Github : https://github.com/haikelJebahi/Graveyard-of-wikidata
Data : https://iccl.inf.tu-dresden.de/web/Wikidata_SPARQL_Logs/en

In [1]:
## Imports

In [2]:
import csv
import pandas as pd
from urllib.parse import unquote_plus
from pyparsing import ParseException
from rdflib.plugins.sparql.algebra import translateQuery, translateUpdate, pprintAlgebra, traverse
from rdflib.plugins.sparql.parser import parseQuery, parseUpdate
from rdflib.plugins.sparql.operators import Builtin_LANG
from rdflib.plugins.sparql.parserutils import CompValue
from rdflib.term import Variable
import re
from pandas import concat

In [3]:
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.expand_frame_repr', True)
pd.options.display.max_colwidth = 0

## Import parsed CSV

In [4]:
nom="../data/merged/status500_Joined.parsed.tsv"
df_nona = pd.read_csv(nom,sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '../data/merged/status500_Joined.parsed.tsv'

## Getting basic attributes of queries

In [ ]:
df_nona["has_variable"] = df_nona["counter_variable"].apply(lambda x: False if x == 0 else True)
df_nona["has_filter"] = df_nona["counter_filter"].apply(lambda x: False if x == 0 else True)
df_nona["has_orderby"] = df_nona["counter_orderby"].apply(lambda x: False if x == 0 else True)
df_nona["has_limit"] = df_nona["counter_limit"].apply(lambda x: False if x == 0 else True)
df_nona["has_select"] = df_nona["counter_select"].apply(lambda x: False if x == 0 else True)
df_nona["has_distinct"] = df_nona["counter_distinct"].apply(lambda x: False if x == 0 else True)
df_nona["has_modify"] = df_nona["counter_modify"].apply(lambda x: False if x == 0 else True)
df_nona["has_tomultiset"] = df_nona["counter_tomultiset"].apply(lambda x: False if x == 0 else True)
df_nona["has_join"] = df_nona["counter_join"].apply(lambda x: False if x == 0 else True)
df_nona["has_bgp"] = df_nona["counter_bgp"].apply(lambda x: False if x == 0 else True)
df_nona["has_insertdata"] = df_nona["counter_insertdata"].apply(lambda x: False if x == 0 else True)
df_nona["has_deletedata"] = df_nona["counter_deletedata"].apply(lambda x: False if x == 0 else True)
df_nona["has_project"] = df_nona["counter_project"].apply(lambda x: False if x == 0 else True)
df_nona["has_union"] = df_nona["counter_union"].apply(lambda x: False if x == 0 else True)
df_nona["has_values"] = df_nona["counter_values"].apply(lambda x: False if x == 0 else True)
df_nona["has_groupby"] = df_nona["counter_groupby"].apply(lambda x: False if x == 0 else True)
df_nona["has_slice"] = df_nona["counter_slice"].apply(lambda x: False if x == 0 else True)
df_nona["has_offset"] = df_nona["counter_offset"].apply(lambda x: False if x == 0 else True)

Split in robotic and organic

In [ ]:

df_nona_robotic = df_nona.loc[(df_nona['sourceCategory'] == 'robotic')]
df_nona_robotic

In [ ]:
df_nona_organic = df_nona.loc[(df_nona['sourceCategory'] == 'organic')]

In [ ]:
df_nona_sort_by_triples = df_nona.sort_values(["counter_triples"],  ascending=[False])
df_nona_sort_by_triples_out = df_nona_sort_by_triples.head(10)
df_nona_sort_by_triples_out.to_csv("../out/queries_sort_by_triples.tsv", sep='\t', index=False)

## Statistics

In [ ]:
len(df_nona)

### Organic

Number of organic queries

In [ ]:
len(df_nona_organic)

Sum of all variables

In [ ]:
sumVariable = len(df_nona_organic[df_nona_organic["has_variable"]==True])
sumFilter = len(df_nona_organic[df_nona_organic["has_filter"]==True])
sumOrderBy = len(df_nona_organic[df_nona_organic["has_orderby"]==True])
sumLimit = len(df_nona_organic[df_nona_organic["has_limit"]==True])
sumSelect = len(df_nona_organic[df_nona_organic["has_select"]==True])
sumDistinct = len(df_nona_organic[df_nona_organic["has_distinct"]==True])
sumToMultiSet = len(df_nona_organic[df_nona_organic["has_tomultiset"]==True])
sumJoin = len(df_nona_organic[df_nona_organic["has_join"]==True])
sumBGP = len(df_nona_organic[df_nona_organic["has_bgp"]==True])
sumproject = len(df_nona_organic[df_nona_organic["has_project"]==True])
sumUnion = len(df_nona_organic[df_nona_organic["has_union"]==True])
sumValues = len(df_nona_organic[df_nona_organic["has_values"]==True])
sumSlice = len(df_nona_organic[df_nona_organic["has_slice"]==True])
sumSlice = len(df_nona_organic[df_nona_organic["has_offset"]==True])
sumGroupBy = len(df_nona_organic[df_nona_organic["has_groupby"]==True])
sumOffsetR = len(df_nona_organic[df_nona_organic["has_offset"]==True])
#sumModify = df_nona_organic["counter_modify"].sum() # TODO
#sumInsertDate = df_nona_organic["counter_insertdata"].sum() #TODO
#sumDeleteData = df_nona_organic["counter_deletedata"].sum()  #TODO

sumVariableR = len(df_nona_robotic[df_nona_robotic["has_variable"]==True])
sumFilterR = len(df_nona_robotic[df_nona_robotic["has_filter"]==True])
sumOrderByR = len(df_nona_robotic[df_nona_robotic["has_orderby"]==True])
sumLimitR = len(df_nona_robotic[df_nona_robotic["has_limit"]==True])
sumSelectR = len(df_nona_robotic[df_nona_robotic["has_select"]==True])
sumDistinctR = len(df_nona_robotic[df_nona_robotic["has_distinct"]==True])
sumToMultiSetR = len(df_nona_robotic[df_nona_robotic["has_tomultiset"]==True])
sumJoinR = len(df_nona_robotic[df_nona_robotic["has_join"]==True])
sumBGPR = len(df_nona_robotic[df_nona_robotic["has_bgp"]==True])
sumprojectR = len(df_nona_robotic[df_nona_robotic["has_project"]==True])
sumUnionR = len(df_nona_robotic[df_nona_robotic["has_union"]==True])
sumValuesR = len(df_nona_robotic[df_nona_robotic["has_values"]==True])
sumSliceR = len(df_nona_robotic[df_nona_robotic["has_slice"]==True])
sumSliceR = len(df_nona_robotic[df_nona_robotic["has_offset"]==True])
sumGroupByR = len(df_nona_robotic[df_nona_robotic["has_groupby"]==True])
sumOffset = len(df_nona_robotic[df_nona_robotic["has_offset"]==True])
#sumModify = df_nona_organic["counter_modify"].sum() # TODO
#sumInsertDate = df_nona_organic["counter_insertdata"].sum() #TODO
#sumDeleteData = df_nona_organic["counter_deletedata"].sum()  #TODO


da = {
'I7 Robotic': [sumVariableR, sumValuesR, sumFilterR, sumOrderByR, sumLimitR, sumSelectR, sumDistinctR, sumSliceR, sumToMultiSetR,sumJoinR,sumBGPR, sumprojectR, sumUnionR, sumGroupByR, sumOffsetR]
, 'I7 Organic': [sumVariable, sumValues, sumFilter, sumOrderBy, sumLimit, sumSelect, sumDistinct, sumSlice, sumToMultiSet,sumJoin,sumBGP, sumproject, sumUnion, sumGroupBy, sumOffset]
}
dpfr = pd.DataFrame(data=da, index=['Variable','Values','Filter','OrderBy','Limit','Select','Distinct','Slice', 'ToMultiSet','Join','BGP', 'project', 'Union', 'GroupBy', 'Offset'])

dpfr



Average number of variable per organic request

In [ ]:
df_nona_organic["counter_variable"].mean()

### Robotic

In [ ]:
len(df_nona_robotic)

Sum of all variables

In [ ]:
df_nona_robotic["counter_variable"].sum()

In [ ]:
df_nona_robotic["counter_variable"].mean()

In [ ]:
sumVariable = df_nona_robotic["counter_variable"].sum()
 
sumFilter = df_nona_robotic["counter_filter"].sum() 
sumOrderBy = df_nona_robotic["counter_orderby"].sum() 
sumLimit = df_nona_robotic["counter_limit"].sum() 
sumSelect = df_nona_robotic["counter_select"].sum()
sumDistinct = df_nona_robotic["counter_distinct"].sum()
sumModify = df_nona_robotic["counter_modify"].sum() 
sumToMultiSet = df_nona_robotic["counter_tomultiset"].sum() 
sumJoin = df_nona_robotic["counter_join"].sum() 
sumBGP = df_nona_robotic["counter_bgp"].sum() 
sumInsertDate = df_nona_robotic["counter_insertdata"].sum()
sumDeleteData = df_nona_robotic["counter_deletedata"].sum() 
sumproject = df_nona_robotic["counter_project"].sum() 
sumUnion = df_nona_robotic["counter_union"].sum() 
sumValues = df_nona_robotic["counter_values"].sum() 

sumOffset = df_nona_robotic["counter_offset"].sum() 
sumGroupBy = df_nona_robotic["counter_groupby"].sum() 



da = {'Robotic': [sumVariable, sumValues, sumFilter, sumOrderBy, sumLimit, sumSelect, sumDistinct, sumModify, sumToMultiSet,sumJoin,sumBGP,sumInsertDate,sumDeleteData, sumproject, sumUnion, sumGroupBy, sumOffset]}
dpfr = pd.DataFrame(data=da, index=['Variable','Values','Filter','OrderBy','Limit','Select','Distinct','Modify', 'ToMultiSet','Join','BGP','InsertDate','DeleteData', 'project', 'Union', 'GroupBy', 'Offset'])

dpfr

In [ ]:
df_nona_robotic["counter_variable"].mean()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=513ed372-d958-4618-b5dc-4b186f2555bd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>